In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("key-value_rdd_op_joins")
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 04:06:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Operations

- `groupByKey`
    - `KeyValueRDD.groupByKey()`
    - 그룹핑 후에 특정 Transformations 같은 연산
    - `Key` 값이 있는 상태에서 시작
- `groupBy()`
    - `RDD.groupBy(numPartitions=None, partitionFunc=<function portable_hash>)`
    - 함수에 의해서 그룹이 생기는 연산

In [2]:
rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

In [3]:
g_rdd = rdd.groupByKey()
g_rdd.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x7f573060a6d0>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x7f573060a7c0>)]

In [4]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [5]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

In [6]:
# groupBy 사용하기
rdd = sc.parallelize([
    "C", "C++", "C#", "Python", "Java", "JavaScript"
])

In [7]:
# groupBy는 그룹핑할 키에 대한 정의를 개발자가 직접 해준다.
grouped = rdd.groupBy(lambda x : x[0])
grouped.glom().collect()

[[('C', <pyspark.resultiterable.ResultIterable at 0x7f5730624b50>),
  ('P', <pyspark.resultiterable.ResultIterable at 0x7f57306270d0>),
  ('J', <pyspark.resultiterable.ResultIterable at 0x7f5730627130>)]]

In [8]:
grouped.mapValues(list).collect()

[('C', ['C', 'C++', 'C#']), ('P', ['Python']), ('J', ['Java', 'JavaScript'])]

In [10]:
grouped.getNumPartitions()

1

In [11]:
g_rdd.getNumPartitions()

1

In [12]:
# groupByKey는 K-V RDD를 사용할 때 Key가 알아서 그룹핑의 기준이 된다.

x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

y = x.groupByKey()

In [13]:
y.collect()

[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f5730446640>),
 ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f5730457a60>),
 ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f5730479910>)]

In [14]:
y.glom().collect()

[[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f57304a6b80>),
  ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f5730511700>),
  ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f57305115b0>)],
 [],
 []]

# Joins

In [15]:
# Inner Join : 서로간에 존재하는 키만 합쳐줍니다.
rdd1 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("hoo", 3)
])

rdd2 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("goo", 10),
    ("moo", 6),
])

rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

**Outer Join**
- 기준이 되는 한 쪽에는 데이터가 있고, 다른 쪽에는 데이터가 없는 경우
    - 설정한 기준에 따라서 기준에 맞는 데이터가 항상 남아있는다.
- `leftOuterJoin` : 왼쪽에 있는 rdd가 기준이 됩니다. (함수를 호출하는 쪽)
- `rightOuterJoin` : 오른쪽에 있는 rdd가 기준이 됩니다. (함수에 매개변수로 들어가는 쪽)

In [16]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [17]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [18]:
sc.stop()